## Notes on Conversion from Lat/Lon Coordinates to UTM coordinates

1. Lat/Lon given with uniform spacing in terms of degrees. This is misleading because this does not correspond to uniform spacing spatially. (without uniform spacing spatially it is difficult to compute gradients / orientations) 
2. There is a nonlinearity in the mapping between Lat/Lon and UTM northings and eastings. Given a constant value for Lat (with varying Lon), both the easting and northing change (instead of just easting). This makes the conversion to uniform spacing non-trivial because you cannot just "stretch" the bathymetry array
3. It is inefficient to store all values in the resulting UTM coordinate matrix. Since the bathymetry is provided as 10m gridding, the input (dense) matrix is ~$2*10^6$ entries. For the full spatial extent in UTM with 1m spacing, the sparse matrix would be of size ~$2*10^8$ entries, or a factor of 100 times larger than the input -- this is computationally inefficient. This motivates the use of a pandas DataFrame instead of using default numpy array because Pandas DataFrame you explicitly keep track of x-coordinate and y-coordinate as seperate columns (still sparse) and still have ability to perform filtering on features (such as on depth/slope/orientation/position)


# Plotting Bathymetric Data in Python

In [28]:
import seaborn as sns 
import numpy as np
import datetime
import rasterio as rio
import importlib
import pandas as pd
# import earthpy as et
# import earthpy.plot as ep
import scipy
import scipy.signal
import os
import sys
import utm

sys.path.insert(1, os.path.join(sys.path[0], '../data'))
sys.path.append(os.path.join(sys.path[0], '../'))

import BathymetryMap
import bathy_meta_data
from matplotlib import pyplot as plt 
from PIL import Image
def reload_modules():
    importlib.reload(bathy_meta_data)
    importlib.reload(BathymetryMap)

## Load Data

In [48]:
reload_modules()
meta_dict = bathy_meta_data.BathyData["Kolumbo_full_AR"]
# meta_dict = bathy_meta_data.BathyData["Kolumbo"]
# meta_dict = bathy_meta_data.BathyData["Kolumbo_full"]
bathy     = BathymetryMap.BathymetryMap(meta_dict=meta_dict)

bathy.parse_bathy_file()

print(bathy.left, bathy.right, bathy.top, bathy.bottom)

delta_lon = (bathy.right - bathy.left)/bathy.width
delta_lat = (bathy.top   - bathy.bottom)/bathy.height

delta_lon, delta_lat

25.366617000001263 25.516716996209418 36.633349996631296 36.49995000000126


(0.00011184798525197843, 9.019607615282796e-05)

---
## Data Parsing 

In [49]:
x_coords = []
y_coords = [] 

corners =  [
    (bathy.top,    bathy.left), 
    (bathy.bottom, bathy.left), 
    (bathy.bottom, bathy.right),
    (bathy.top,    bathy.right)
]

for (lat,lon) in corners:
    x,y,_,_ = utm.from_latlon(lat,lon)
    x_coords.append(int(np.round(x)))
    y_coords.append(int(np.round(y)))

x_min = min(x_coords)
x_max = max(x_coords)
y_min = min(y_coords)
y_max = max(y_coords)
x_rng = x_max - x_min 
y_rng = y_max - y_min
print(x_rng)
print(y_rng)
print(x_rng*y_rng)

13672
15017
205312424


## Calculate Variance of Factor Maps

In [50]:
# if M is (nxn), function returns a (nxn) matrix where each point represents the variance of the original point based on 8 surrounding points
def calculateVar_pts_in_matrix(M):
    var_matrix = np.zeros((len(M[:,0]), len(M[0,:])))
    for i in range(1,len(M[:,0])-1):       #for all rows except first and last
        for j in range(1,len(M[0,:])-1):   #for all columns except first and last
            pts = np.array([ M[i,j] , M[i+1, j] , M[i+1, j-1], M[i+1,j+1], M[i,j+1], M[i, j-1], M[i-1, j], M[i-1, j+1], M[i-1, j-1]])
            var_matrix[i,j] = np.var(pts)
    #Set Equal edges to inside rows
    var_matrix[:,0] = var_matrix[:,1]
    var_matrix[:,len(M[0,:])-1] = var_matrix[:,len(M[0,:]) - 2]
    var_matrix[0,:] = var_matrix[1,:]
    var_matrix[len(M[:,0])-1, :] = var_matrix[len(M[:,0]) - 2,:]
    return(var_matrix)

Orientation converted to radians before calculating variance because otherwise the range of -180 deg to 180 deg results in extremely large variances

In [52]:
depth_var = calculateVar_pts_in_matrix(bathy.depth)
slope_var = calculateVar_pts_in_matrix(bathy.slope)
orient_var = calculateVar_pts_in_matrix(bathy.orient*np.pi/180)



In [53]:
print(np.shape(bathy.depth))
print(np.shape(bathy.slope))
print(np.shape(bathy.orient))
print(np.shape(depth_var))
print(np.shape(slope_var))
print(np.shape(orient_var))

(1479, 1342)
(1479, 1342)
(1479, 1342)
(1479, 1342)
(1479, 1342)
(1479, 1342)


## Package and Save Data

In [54]:
utm_x_list      = []
utm_y_list      = [] 
depth_list      = []
slope_list      = []
orient_list     = []

depth_var_list  = []
slope_var_list  = []
orient_var_list = []

# j = 0       -->  top side 
# j = height  -->  bottom side
# i = 0       -->  left side 
# i = width   -->  right side
for i in range(bathy.width):
    for j in range(bathy.height):
        # extract relevant variables from the bathymetry array
        lat = bathy.top  + ((bathy.bottom-bathy.top) /bathy.height)*j
        lon = bathy.left + ((bathy.right -bathy.left)/bathy.width) *i
        utm_x, utm_y, _, _ = utm.from_latlon(lat, lon)
        cur_depth  = bathy.depth[j, i]
        cur_slope  = bathy.slope[j, i]
        cur_orient = bathy.orient[j, i]
        
        cur_depth_var  = depth_var[j, i]
        cur_slope_var  = slope_var[j, i]
        cur_orient_var = orient_var[j, i]
        
        utm_x_list.append(utm_x)
        utm_y_list.append(utm_y)
        depth_list.append(cur_depth)
        slope_list.append(cur_slope)
        orient_list.append(cur_orient)
        depth_var_list.append(cur_depth_var)
        slope_var_list.append(cur_slope_var)
        orient_var_list.append(cur_orient_var)
#     print(i/bathy.width)

In [55]:
bathy_data = {
    'utm_x_list'  : utm_x_list,
    'utm_y_list'  : utm_y_list,
    'depth_list'  : depth_list,
    'slope_list'  : slope_list,
    'orient_list' : orient_list,
    'depth_var'   : depth_var_list,
    'slope_var'   : slope_var_list,
    'orient_var'  : orient_var_list,
}
bathy_df = pd.DataFrame.from_dict(bathy_data)

In [56]:
y_min = 4.041e6
y_max = 4.051e6
x_min = 3.610e5
x_max = 3.670e5


bathy_sub_df = bathy_df[(bathy_df.utm_x_list < x_max) & 
                        (bathy_df.utm_x_list > x_min) &
                        (bathy_df.utm_y_list < y_max) & 
                        (bathy_df.utm_y_list > y_min)]

# ts.df[(ts.df.btm_beam0_range < danger) & 
#     (ts.df.btm_beam1_range < danger) & 
#     (ts.df.btm_beam2_range < danger) & 
#     (ts.df.btm_beam3_range < danger)]

In [57]:
bathy_sub_df

,utm_x_list,utm_y_list,depth_list,slope_list,orient_list,depth_var,slope_var,orient_var
1052002,361007.683779,4.050991e+06,422.056488,2.577596,119.823788,0.143489,0.479790,0.817928
1052003,361007.521966,4.050981e+06,422.033447,2.973290,58.863380,0.166843,0.620163,0.752791
1052004,361007.360153,4.050971e+06,421.443512,2.725101,51.267164,0.148585,0.635052,1.714364
1052005,361007.198340,4.050961e+06,421.315369,1.403043,122.549864,0.063859,0.421853,3.013535
1052006,361007.036528,4.050951e+06,421.766663,1.529471,175.797249,0.058572,0.803067,4.013524
...,...,...,...,...,...,...,...,...
1961092,366999.003728,4.041048e+06,326.323151,2.023454,78.321212,0.073980,0.041654,0.083269
1961093,366998.849396,4.041038e+06,326.193787,1.963412,63.829211,0.079084,0.057781,0.024612
1961094,366998.695064,4.041028e+06,325.997650,1.618389,63.812913,0.059650,0.062958,0.057362
1961095,366998.540733,4.041018e+06,325.924530,1.530675,90.276213,0.047199,0.037224,0.157944


In [ ]:
# bathy_df.copy().append(pd.DataFrame.from_dict(
#     {'depth_list'  : 0, 
#      'orient_list' : 0, 
#      'utm_x_list'  : 0, 
#      'utm_y_list'  : 0, 
#      'orient_list' : 0}))

# bathy_df.copy().append(pd.DataFrame([[0,0,0,0,0]], columns=bathy_df.columns))
# bathy_df

In [58]:
bathy_sub_df.to_csv("C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/bathy/Kolumbo-10m-sub-withVar-orientInRad.csv")
# 'C:/Users/grego/Dropbox/Kolumbo cruise 2019/zduguid/bathy/Kolumbo-10m-utm-sub.csv'

## Plotting

In [ ]:
fig,ax=plt.subplots(figsize=(10,10))
sns.scatterplot(
    bathy_sub_df.utm_x_list,
    bathy_sub_df.utm_y_list,
    bathy_sub_df.slope_list,
#     palette='viridis_r',
    palette='inferno_r',
#     palette='twilight_shifted',
    linewidth=0,
    legend=False)
plt.axis('equal')
#plt.savefig('/Users/zduguid/Desktop/fig/tmp.png')
#plt.close()

In [ ]:
# bathy.plot_depth_map()
# bathy.plot_slope_map()
# bathy.plot_orientation_map()
# bathy.plot_three_factors()
bathy.plot_depth_contours()

In [ ]:
# # plot seafloor depth
# sns.set(font_scale = 1.5)
# fig, ax = plt.subplots(figsize=(15,8))
# ep.plot_bands(
#     -bathy, 
#     cmap='viridis_r',
#     title="Depth Map [m]",
#     ax=ax,
#     scale=False
# )

In [ ]:
def get_utm_coords_from_bathy_lat_lon(lat, lon): 
    """TODO
    """
    utm_pos  = utm.from_latlon(lat, lon)
    easting  = round(utm_pos[0],2)
    northing = round(utm_pos[1],2)
    zone     = utm_pos[2]
    return(easting, northing, zone)

get_utm_coords_from_bathy_lat_lon(bathy_l, bathy_b)

# bathy_l, bathy_r, bathy_b, bathy_t

In [ ]:

bathy[bathy>=0]=np.nan
bathy.shape

In [ ]:
scharr = np.array([[ +3 -3j,  +10    , +3 +3j],
                   [    -10j,   0    ,    +10j],
                   [ -3 -3j,  -10    , -3 +3j]]) / 32  # Gx + j*Gy

In [ ]:
bathy_grad = scipy.signal.convolve2d(bathy, scharr, boundary='symm', mode='same')/(np.max([delta_x,delta_y]))

In [ ]:
bathy_slope  = np.arctan(np.absolute(bathy_grad))*RAD_TO_DEG

In [ ]:
bathy_orient = np.angle(bathy_grad)*RAD_TO_DEG

## Plot Environment Data

In [ ]:
# sns.set(font_scale = 1.5)
# fig, ax = plt.subplots(2,3, figsize=(15,8), gridspec_kw={'height_ratios': [2, 1]})
# fig.subplots_adjust(hspace=.4)
# fig.subplots_adjust(wspace=.3)
# plt.suptitle(data['title'], fontweight='bold', fontsize=28)


# # plot seafloor depth
# ep.plot_bands(
#     -bathy, 
#     cmap='viridis_r',
#     title="Depth Map [m]",
#     ax=ax[0,0],
#     scale=False
# )


# # plot seafloor slope
# ep.plot_bands(
#     bathy_slope, 
#     cmap='inferno_r',
#     title='Gradiant [deg]',
#     ax=ax[0,1],
#     scale=False
# )


# # plot seafloor slope orientation 
# ep.plot_bands(
#     bathy_orient, 
#     cmap='twilight_shifted',
#     title='Orientation [deg]',
#     ax=ax[0,2],
#     scale=False
# )


# sns.kdeplot(-bathy.flatten(),       shade=True, ax=ax[1,0], linewidth=3)
# sns.kdeplot(bathy_slope.flatten(),  shade=True, ax=ax[1,1], linewidth=3)
# sns.kdeplot(bathy_orient.flatten(), shade=True, ax=ax[1,2], linewidth=3)
# ax[1,0].set_xlabel('Depth [m]')
# ax[1,1].set_xlabel('Slope [deg]')
# ax[1,2].set_xlabel('Orientation [deg]')
# xticks  = np.arange(bathy_w/data['num_ticks']/2, bathy_w, bathy_w/data['num_ticks'])
# xlabels = [data['ticks'] % np.round((bathy_r-bathy_l)*(i/bathy_w) + bathy_l, 5) for i in xticks]
# yticks  = np.arange(bathy_h/data['num_ticks']/2, bathy_h, bathy_h/data['num_ticks'])
# ylabels = [data['ticks'] % np.round((bathy_b-bathy_t)*(i/bathy_h) + bathy_t, 5) for i in yticks]


# # plot axis labels 
# for i in range(3):
#     ax[0,i].set_xticks(xticks)
#     ax[0,i].set_xticklabels(xlabels)
#     ax[0,i].set_yticks(yticks)
#     if i==0:
#         ax[0,i].set_yticklabels(ylabels)
#         ax[0,i].set_ylabel(data['ylabel'])
#         ax[1,i].set_ylabel('Kernel Density')
#     else:
#         ax[0,i].set_yticklabels(['' for _ in ylabels])
#     ax[0,i].set_xlabel(data['xlabel'])
#     ax[0,i].grid(linewidth=1, alpha=1, color='lavenderblush')
# plt.savefig('/Users/zduguid/Desktop/fig/tmp.png')
# print('>> Plotting Complete')
# plt.show()